In [65]:
import spacy
from spacy import displacy
from spacy.tokens import DocBin
import json
from sklearn.model_selection import train_test_split

In [66]:
data = json.load(open("annotations.json"))

len(data)

48

In [67]:
def spacify(data):
    nlp = spacy.blank("id")
    db = DocBin()

    for text, annot in data:
        doc = nlp.make_doc(text)
        annot = annot["entities"]

        ents, ent_indices = [], []

        for start, end, label in annot:
            skip_entity = False
            for i in range(start, end):
                if i in ent_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            ent_indices = ent_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
            except:
                continue

            if span is None:
                err = str([start, end]) + "  " + str(text) + "\n"
                print(err)
            else:
                ents.append(span)
        
    try:
        doc.ents = ents
        db.add(doc)
    except:
        pass

    return db

In [68]:
train_data, test_data = train_test_split(data, test_size=0.2)
len(train_data), len(test_data)

(38, 10)

In [69]:
db = spacify(train_data)
db.to_disk("train_data.spacy")

db = spacify(test_data)
db.to_disk("test_data.spacy")

```cmd
py -m spacy train config.cfg --output output --paths.train train_data.spacy --paths.dev test_data.spacy --gpu-id 0
```

In [70]:
text = "cari kustomer yg dari cabang tomang"

nlp = spacy.load("output/model-best")
doc = nlp(text)

for ent in doc.ents:
    print((ent.text, ent.label_))

c:\Users\paoma\miniconda3\lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fi